# Configure the Gremlin server
You must run the below before starting.

In [ ]:
%%capture output
%%graph_notebook_config
{
    "host": "127.0.0.1",
    "port": 8182,
    "ssl": false,
    "gremlin" : {
       "message_serializer": "graphbinary"
    }
}

# Basic Terminology and Background

The following is some information on some concepts that are generally confusing for new gremlin users.

## TinkerPop

TinkerPop is a graph computer framework. This includes the Gremlin query language, Gremlin server, all the gremlin drivers, and other things.

## Gremlin

Gremlin is a graph query language. Queries are often called traversals. Gremlin is a functional language, meaning that queries are composed of functions that are chained together.

If you are familiar with stream processing, Gremlin is similar to that. A traversal is a stream of data that is processed by a series of functions.

## Driver Remote Connection

TinkerPop has a number of drivers, these usually have a common theme of having a DriverRemoteConnection that is used to connect to the database. Different languages have slightly different API's, but this theme is faily common. The DriverRemoteConnection spawns `g` which is used to execute the traversal.

## What is 'g'?

I keep seeing 'g' everywhere, what is it? g is a GraphTraversalSource and it spawns a GraphTraversal when it starts. When this GraphTraversal is terminated, the driver (DriverRemoteConnection) sends it to the server for execution.

# Gremlin 100

## Start by clearing any data in the graph
Start by removing all vertices and edges from the graph.
If you remove a vertex, all edges connected to that vertex are also implicitly removed.
Running g.V() gets all vertices, running drop() after removes all of them.

In [ ]:
%%gremlin

g.V().drop().iterate()

return "Sucessfully dropped graph"

### What is .iterate()?
Gremlin traversals need to be terminated. This is done by calling iterate(), next(), or toList().
This tells the driver to execute everything up to this point in the traversal.
.iterate() => Return nothing
.next() => Return the next element in the traversal
.toList() => Return all elements in the traversal as a list
We will revisit this later, for now lets add some data

## Adding data
To add a vertex, use addV()
Vertices has 1 label, properties, an id, and edges.
Let's add Lyndon, Grant, and Simon

In [ ]:
%%gremlin

g.addV("Person").property("name", "Lyndon").property("age", 30).iterate()
g.addV("Person").property("name", "Grant").property("age", 50).iterate()
g.addV("Person").property("name", "Simon").property("age", 12).iterate()

return "Sucessfully added data"

## Reading the data back
We can now look up the vertices we just created
Note we use next() when we want a single person, toList() when we want multiple.
You can call next() on what would be a list of data, you'd just get the first item.

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").next()

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Grant").next()

In [ ]:
%%gremlin

// Note there is a serializer issue with graphbinary and the client underneath this notebook so g.V().hasLabel("Person") will fail with a type error. This is a result of some changes to the tinkerpop serializers that the notebook community has not caught up with yet.
g.V().hasLabel("Person").values("name")


## Visualize our simple graph
By returning a path we can visualize the graph. Run the query and hit the 'Graph' tab. Notice we have no edges so there's just 3 separate vertices.
Click a vertex and hit the button with the 3 horizontal lines to see more information.

Note the query must return data for this to have information (we path via elementMap() to get all data).

In [ ]:
%%gremlin --edge-label-max-length 30 --label-max-length 30 -p v

g.V().path().by(elementMap())

## Add some edges to the graph
Let's add some edges. Let's make Lyndon know Grant and Simon, and Grant know Simon and Lyndon, but Simon not know anyone.

In [ ]:
%%gremlin

// Notice the traversal inside the traversal. We are adding an edge from the result of V().hasLabel("Person").has("name", "Lyndon")
// to the result of V().hasLabel("Person").has("name", "Grant").
// If either of these returned >1 result we'd add >1 edge, but we know the structure of our graph and that these are singular vertices.
g.addE("knows").
    from(V().hasLabel("Person").has("name", "Lyndon")).
    to(V().hasLabel("Person").has("name", "Grant")).
    iterate()
g.addE("knows").
    from(V().hasLabel("Person").has("name", "Lyndon")).
    to(V().hasLabel("Person").has("name", "Simon")).
    iterate()
g.addE("knows").
    from(V().hasLabel("Person").has("name", "Grant")).
    to(V().hasLabel("Person").has("name", "Simon")).iterate()
g.addE("knows").
    from(V().hasLabel("Person").has("name", "Grant")).
    to(V().hasLabel("Person").has("name", "Lyndon")).iterate()

return "Successfully added edges"

## Visualize our Graph
Let's run the visualization again, this time we will have edges.

In [ ]:
%%gremlin --edge-label-max-length 30 --label-max-length 30 -p v,oute,inv

g.V().outE().inV().path().by(elementMap())

## Traverse edges

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").outE().count()

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").inE().toList()

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").out().count()

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").in().toList()

In [ ]:
%%gremlin

g.V().hasLabel("Person").has("name", "Lyndon").out("knows").has("name", "Grant").toList()